In [3]:
import numpy as np

# Gaussian blur kernel for pyramids
gaussian_kernel = (
    np.array(
        [
            [1,  4,  6,  4, 1],
            [4, 16, 24, 16, 4],
            [6, 24, 36, 24, 6],
            [4, 16, 24, 16, 4],
            [1,  4,  6,  4, 1]
        ]
    ) / 256
)

# YIQ and RGB color space conversion matrices
yiq_from_rgb = np.array([
    [0.29900000,  0.58700000,  0.11400000],
    [0.59590059, -0.27455667, -0.32134392],
    [0.21153661, -0.52273617,  0.31119955]
], dtype=np.float32)

rgb_from_yiq = np.linalg.inv(yiq_from_rgb)


In [4]:
import cv2
import numpy as np
import tqdm


def loadVideo(video_path):
    image_sequence = []
    video = cv2.VideoCapture(video_path)
    fps = video.get(cv2.CAP_PROP_FPS)
    while video.isOpened():
        ret, frame = video.read()
        if not ret:
            break
        image_sequence.append(frame[:, :, ::-1])  # Convert BGR to RGB
    video.release()
    return np.asarray(image_sequence), fps

def rgb2yiq(rgb_image):
    return rgb_image.astype(np.float32) @ yiq_from_rgb.T

def yiq2rgb(yiq_image):
    return yiq_image.astype(np.float32) @ rgb_from_yiq.T

def pyrDown(image, kernel):
    return cv2.filter2D(image, -1, kernel)[::2, ::2]

def pyrUp(image, kernel, dst_shape=None):
    dst_height = image.shape[0] + 1
    dst_width = image.shape[1] + 1
    if dst_shape is not None:
        dst_height -= (dst_shape[0] % image.shape[0] != 0)
        dst_width -= (dst_shape[1] % image.shape[1] != 0)
    upsampled_image = np.insert(image, np.arange(1, dst_height), 0, axis=0)
    upsampled_image = np.insert(upsampled_image, np.arange(1, dst_width), 0, axis=1)
    return cv2.filter2D(upsampled_image, -1, 4 * kernel)

def idealTemporalBandpassFilter(images, fps, freq_range, axis=0):
    fft = np.fft.fft(images, axis=axis)
    frequencies = np.fft.fftfreq(images.shape[0], d=1.0 / fps)
    low = (np.abs(frequencies - freq_range[0])).argmin()
    high = (np.abs(frequencies - freq_range[1])).argmin()
    fft[:low] = 0
    fft[high:] = 0
    return np.fft.ifft(fft, axis=0).real

def reconstructGaussianImage(image, pyramid):
    yiq = rgb2yiq(image) + pyramid
    rgb = yiq2rgb(yiq)
    return np.clip(rgb, 0, 255).astype(np.uint8)

def reconstructLaplacianImage(image, pyramid, kernel):
    reconstructed_image = rgb2yiq(image)
    for level in range(1, pyramid.shape[0] - 1):
        tmp = pyramid[level]
        for curr_level in range(level):
            tmp = pyrUp(tmp, kernel, pyramid[level - curr_level - 1].shape[:2])
        reconstructed_image += tmp.astype(np.float32)
    return np.clip(yiq2rgb(reconstructed_image), 0, 255).astype(np.uint8)

def getGaussianOutputVideo(original_images, filtered_images):
    video = np.zeros_like(original_images)
    for i in tqdm.tqdm(range(filtered_images.shape[0]), desc="Reconstructing Video", ascii=True):
        video[i] = reconstructGaussianImage(original_images[i], filtered_images[i])
    return video

def getLaplacianOutputVideo(original_images, filtered_images, kernel):
    video = np.zeros_like(original_images)
    for i in tqdm.tqdm(range(original_images.shape[0]), desc="Reconstructing Video", ascii=True):
        video[i] = reconstructLaplacianImage(original_images[i], filtered_images[i], kernel)
    return video

def saveVideo(video, saving_path, fps):
    height, width = video[0].shape[:2]
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    writer = cv2.VideoWriter(saving_path, fourcc, fps, (width, height))
    for i in tqdm.tqdm(range(len(video)), desc="Saving Video", ascii=True):
        writer.write(video[i][:, :, ::-1])  # Convert RGB to BGR
    writer.release()


In [5]:
import numpy as np
import tqdm


def generateGaussianPyramid(image, kernel, level):
    image_shape = [image.shape[:2]]
    downsampled_image = image.copy()
    for _ in range(level):
        downsampled_image = pyrDown(downsampled_image, kernel)
        image_shape.append(downsampled_image.shape[:2])
    gaussian_pyramid = downsampled_image
    for curr_level in range(level):
        gaussian_pyramid = pyrUp(
            gaussian_pyramid,
            kernel=kernel,
            dst_shape=image_shape[level - curr_level - 1]
        )
    return gaussian_pyramid

def getGaussianPyramids(images, kernel, level):
    gaussian_pyramids = np.zeros_like(images, dtype=np.float32)
    for i in tqdm.tqdm(range(images.shape[0]), desc="Gaussian Pyramids Generation", ascii=True):
        yiq_image = rgb2yiq(images[i])
        gaussian_pyramids[i] = generateGaussianPyramid(yiq_image, kernel, level)
    return gaussian_pyramids

def filterGaussianPyramids(pyramids, fps, freq_range, alpha, attenuation):
    filtered_pyramids = idealTemporalBandpassFilter(
        images=pyramids,
        fps=fps,
        freq_range=freq_range
    ).astype(np.float32)
    filtered_pyramids *= alpha
    filtered_pyramids[:, :, :, 1:] *= attenuation
    return filtered_pyramids


In [6]:
import numpy as np
import tqdm


def generateLaplacianPyramid(image, kernel, level):
    laplacian_pyramid = []
    prev_image = image.copy()
    for _ in range(level):
        downsampled = pyrDown(prev_image, kernel)
        upsampled = pyrUp(downsampled, kernel, dst_shape=prev_image.shape[:2])
        laplacian = prev_image - upsampled
        laplacian_pyramid.append(laplacian)
        prev_image = downsampled
    return laplacian_pyramid

def getLaplacianPyramids(images, kernel, level):
    laplacian_pyramids = []
    for image in tqdm.tqdm(images, desc="Laplacian Pyramids Generation", ascii=True):
        yiq_image = rgb2yiq(image)
        pyramid = generateLaplacianPyramid(yiq_image, kernel, level)
        laplacian_pyramids.append(pyramid)
    return np.asarray(laplacian_pyramids, dtype='object')

def filterLaplacianPyramids(pyramids, level, fps, freq_range, alpha, lambda_cutoff, attenuation):
    filtered = np.zeros_like(pyramids)
    delta = lambda_cutoff / (8 * (1 + alpha))
    b_low, a_low = butter(1, freq_range[0], btype='low', fs=fps)
    b_high, a_high = butter(1, freq_range[1], btype='low', fs=fps)
    lowpass = pyramids[0]
    highpass = pyramids[0]
    filtered[0] = pyramids[0]
    for i in tqdm.tqdm(range(1, pyramids.shape[0]), desc="Laplacian Pyramids Filtering", ascii=True):
        lowpass = (-a_low[1] * lowpass + b_low[0] * pyramids[i] + b_low[1] * pyramids[i - 1]) / a_low[0]
        highpass = (-a_high[1] * highpass + b_high[0] * pyramids[i] + b_high[1] * pyramids[i - 1]) / a_high[0]
        filtered[i] = highpass - lowpass
        for lvl in range(1, level - 1):
            height, width, _ = filtered[i, lvl].shape
            lambd = np.sqrt(height ** 2 + width ** 2)
            new_alpha = (lambd / (8 * delta)) - 1
            filtered[i, lvl] *= min(alpha, new_alpha)
            filtered[i, lvl][:, :, 1:] *= attenuation
    return filtered


In [7]:


def gaussian_evm(images, fps, kernel, level, alpha, freq_range, attenuation):
    pyramids = getGaussianPyramids(images, kernel, level)
    print("Gaussian Pyramids Filtering...")
    filtered = filterGaussianPyramids(pyramids, fps, freq_range, alpha, attenuation)
    print("Finished filtering!")
    output = getGaussianOutputVideo(original_images=images, filtered_images=filtered)
    return output

def laplacian_evm(images, fps, kernel, level, alpha, lambda_cutoff, freq_range, attenuation):
    pyramids = getLaplacianPyramids(images, kernel, level)
    filtered = filterLaplacianPyramids(pyramids, level, fps, freq_range, alpha, lambda_cutoff, attenuation)
    output = getLaplacianOutputVideo(original_images=images, filtered_images=filtered, kernel=kernel)
    return output


In [8]:
# === Provide the path to your video ===
video_path = "/content/250 rpm with small angle_clip10_motion.avi"  # <-- Replace with your actual video filename or full path

# === Load the video frames and fps ===
images, fps = loadVideo(video_path)
print(f"Loaded video with {len(images)} frames at {fps:.2f} FPS")

# === Define parameters ===
level = 4
alpha = 100
freq_range = [0.833, 1.0]
attenuation = 1
lambda_cutoff = 1000  # Only used in Laplacian

# === Choose method ===
mode = "gaussian"  # or "laplacian"

# === Run selected EVM ===
if mode == "gaussian":
    output_video = gaussian_evm(
        images=images,
        fps=fps,
        kernel=gaussian_kernel,
        level=level,
        alpha=alpha,
        freq_range=freq_range,
        attenuation=attenuation
    )
else:
    output_video = laplacian_evm(
        images=images,
        fps=fps,
        kernel=gaussian_kernel,
        level=level,
        alpha=alpha,
        lambda_cutoff=lambda_cutoff,
        freq_range=freq_range,
        attenuation=attenuation
    )

# === Save output ===
output_path = "evm_output.avi"
saveVideo(output_video, output_path, fps)
print(f"Saved output video to {output_path}")


Loaded video with 150 frames at 30.00 FPS


Gaussian Pyramids Generation: 100%|##########| 150/150 [00:04<00:00, 30.66it/s]


Gaussian Pyramids Filtering...
Finished filtering!


Saving Video: 100%|##########| 150/150 [00:01<00:00, 141.14it/s]

Saved output video to evm_output.avi
